# 📝 Lab #2 (Part 1): 통신사 고객 이탈(Churn) 예측 모델 평가

이번 실습에서는 Day 2 커리큘럼의 핵심 데이터셋인 **'Telco Customer Churn'** 데이터를 사용하여, 지금까지 배운 분류 평가지표들을 종합적으로 적용해보겠습니다.

단순한 분류 모델을 하나 만든 뒤, 그 모델의 성능을 다양한 관점에서 깊이 있게 분석하고 해석하는 능력을 기르는 것을 목표로 합니다.

### 과제 목표
1. `Telco Customer Churn` 데이터를 로드하고 기본적인 전처리를 수행합니다.
2. `LogisticRegression` 모델을 훈련시키고 예측을 수행합니다.
3. **혼동 행렬, 정확도, 정밀도, 재현율, F1-score**를 계산하고 각 지표의 의미를 해석합니다.
4. **ROC Curve**를 그리고 **AUC** 점수를 계산하여 모델의 종합적인 성능을 평가합니다.
5. 각 평가지표를 바탕으로 비즈니스 관점의 질문에 답해봅니다.

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
import plotly.express as px
import plotly.graph_objects as go

### 1. 데이터 로드 및 전처리
데이터는 Kaggle의 'Telco Customer Churn' 데이터셋을 사용합니다. `TotalCharges` 컬럼에 숫자처럼 보이지만 공백이 포함된 값이 있어 전처리가 필요합니다.

In [ ]:
# 데이터 로드
url = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
df = pd.read_csv(url)

# TotalCharges를 숫자형으로 변환 (오류가 나는 값은 NaN으로 처리)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# NaN 값을 중앙값(median)으로 채우기
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# 타겟 변수(Churn)를 0과 1로 변환
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# 특성(X)과 타겟(y) 분리
X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn']

# 수치형, 범주형 변수 분리
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### 2. 모델 훈련 (파이프라인 구축)
전처리 단계를 파이프라인으로 묶어 모델을 훈련시킵니다.

In [ ]:
# 전처리 파이프라인 생성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# 로지스틱 회귀 모델과 파이프라인 연결
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(random_state=42))])

# 모델 훈련
model_pipeline.fit(X_train, y_train)

# 예측 수행
y_pred = model_pipeline.predict(X_test)
y_scores = model_pipeline.predict_proba(X_test)[:, 1]

### 3. 평가지표 계산 및 해석
훈련된 모델의 성능을 다양한 지표로 평가합니다.

In [ ]:
# [문제 1] 혼동 행렬(Confusion Matrix)을 계산하고 시각화하세요.
# cm = ?

# fig_cm = px.imshow(?, text_auto=True,
#                   labels=dict(x="Predicted", y="True"),
#                   x=['Not Churn', 'Churn'], y=['Not Churn', 'Churn'],
#                   title="Churn Prediction Confusion Matrix")
# fig_cm.show()


In [ ]:
# [문제 2] 정확도(Accuracy), 정밀도(Precision), 재현율(Recall), F1-Score를 계산하고 출력하세요.
# accuracy = ?
# precision = ?
# recall = ?
# f1 = ?

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")

#### [문제 3] 결과 분석 질문
1.  이 데이터의 이탈(Churn) 클래스 비율은 약 26.5%로 불균형합니다. 위에서 계산된 **정확도(Accuracy)**가 약 80%인데, 이를 보고 "모델이 꽤 괜찮다"고 판단해도 될까요? 그 이유는 무엇인가요?
2.  고객 이탈을 방지하기 위한 마케팅 캠페인(예: 할인 쿠폰 제공)을 진행한다고 가정해봅시다. 이 캠페인은 **'이탈 가능성이 높은 고객'에게만** 보내고 싶습니다. 이 경우, **정밀도(Precision)**와 **재현율(Recall)** 중 어떤 지표를 더 중요하게 고려해야 할까요? 그 이유를 FP, FN의 관점에서 설명해주세요.

### 4. ROC Curve 및 AUC 평가

In [ ]:
# [문제 4] ROC Curve를 계산하고, AUC 점수를 구하여 함께 시각화하세요.
# fpr, tpr, thresholds = ?
# auc_score = ?

# fig_roc = go.Figure()
# fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (AUC = {auc_score:.4f})'))
# fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
# fig_roc.update_layout(title='ROC Curve for Churn Prediction',
#                   xaxis_title='False Positive Rate',
#                   yaxis_title='True Positive Rate')
# fig_roc.show()

#### [문제 5] ROC/AUC 분석 질문
계산된 AUC 점수는 모델의 성능에 대해 무엇을 말해주나요? (예: '매우 우수하다', '준수하다', '랜덤 추측 수준이다' 등) 그 근거를 설명해주세요.